In [ ]:
from pathlib import Path
from typing import Optional, Union

import simuran as smr

from allensdk.brain_observatory.behavior.behavior_project_cache import (
    VisualBehaviorOphysProjectCache,
)
import pandas as pd
import dtale
import matplotlib.pyplot as plt

In [ ]:
data_storage_directory = Path(r"D:\AllenBrainObservatory\ophys_data")

In [ ]:
here = Path(__file__).parent.absolute()
Path.mkdir(here / "figures", exist_ok=True)
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

In [ ]:
def setup_container(table: pd.DataFrame) -> smr.RecordingContainer:
    